In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-03-22 22:59:07.803761: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 22:59:08.045731: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 22:59:08.045774: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 22:59:08.047017: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-22 22:59:08.155056: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-22 22:59:11.115532: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 22:59:11.231706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 22:59:11.231919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)
loaded = tf.keras.models.load_model("Best_models/best_effnetv2L")
loaded.evaluate(test_dataset)

Found 25744 files belonging to 167 classes.
Using 18021 files for training.


2024-03-22 22:59:11.793286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 22:59:11.793505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 22:59:11.793659: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 25744 files belonging to 167 classes.
Using 7723 files for validation.
Found 11545 files belonging to 167 classes.


2024-03-22 22:59:34.641069: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-03-22 22:59:34.815314: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


361/361 [==============================] - 43s 108ms/step - loss: 0.4881 - accuracy: 0.9058


[0.4881444573402405, 0.9057600498199463]

In [4]:
preprocess_input = tf.keras.applications.efficientnet_v2.preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [5]:
base_model = tf.keras.applications.EfficientNetV2L(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False

for layer in base_model.layers:
    print(layer.name)
len(base_model.layers)

input_1
rescaling
stem_conv
stem_bn
stem_activation
block1a_project_conv
block1a_project_bn
block1a_project_activation
block1a_add
block1b_project_conv
block1b_project_bn
block1b_project_activation
block1b_drop
block1b_add
block1c_project_conv
block1c_project_bn
block1c_project_activation
block1c_drop
block1c_add
block1d_project_conv
block1d_project_bn
block1d_project_activation
block1d_drop
block1d_add
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_project_conv
block2a_project_bn
block2b_expand_conv
block2b_expand_bn
block2b_expand_activation
block2b_project_conv
block2b_project_bn
block2b_drop
block2b_add
block2c_expand_conv
block2c_expand_bn
block2c_expand_activation
block2c_project_conv
block2c_project_bn
block2c_drop
block2c_add
block2d_expand_conv
block2d_expand_bn
block2d_expand_activation
block2d_project_conv
block2d_project_bn
block2d_drop
block2d_add
block2e_expand_conv
block2e_expand_bn
block2e_expand_activation
block2e_project_conv
block2e_project_b

1028

In [6]:
from keras.src import regularizers

inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name="input_layer")

x = base_model(inputs, training=False)
x = layers.GlobalAvgPool2D()(x)


x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = layers.Dense(len(bird_classes), activation='softmax', dtype=tf.float32, kernel_regularizer=regularizers.l2(0.005))(x)

model1 = tf.keras.Model(inputs, outputs)

In [7]:
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-l (Function  (None, 7, 7, 1280)        117746848 
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800

In [8]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3) 

In [9]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='Best_models/best_effnetv2L/', verbose=1, save_best_only=True)

history_eff_b0 = model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                        epochs=10,
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 1/10


2024-03-22 20:32:36.863100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-03-22 20:32:36.917864: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-22 20:32:37.921301: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7cd294003530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-22 20:32:37.921333: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-22 20:32:37.929950: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-22 20:32:37.991359: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


564/564 [==============================] - ETA: 0s - loss: 4.0936 - accuracy: 0.1080
Epoch 1: val_loss improved from inf to 2.85393, saving model to Best_models/best_effnetv2L/
564/564 [==============================] - 149s 243ms/step - loss: 4.0936 - accuracy: 0.1080 - val_loss: 2.8539 - val_accuracy: 0.2313 - lr: 0.0010
Epoch 2/10
563/564 [============================>.] - ETA: 0s - loss: 2.7536 - accuracy: 0.2567
Epoch 2: val_loss improved from 2.85393 to 2.21053, saving model to Best_models/best_effnetv2L/
564/564 [==============================] - 136s 242ms/step - loss: 2.7534 - accuracy: 0.2568 - val_loss: 2.2105 - val_accuracy: 0.3506 - lr: 0.0010
Epoch 3/10
563/564 [============================>.] - ETA: 0s - loss: 2.2624 - accuracy: 0.3489
Epoch 3: val_loss improved from 2.21053 to 1.82799, saving model to Best_models/best_effnetv2L/
564/564 [==============================] - 133s 236ms/step - loss: 2.2622 - accuracy: 0.3489 - val_loss: 1.8280 - val_accuracy: 0.4413 - lr: 0.

In [10]:

for layer in base_model.layers[-100:]:
    layer.trainable = True


In [11]:
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_1 False
rescaling False
stem_conv False
stem_bn False
stem_activation False
block1a_project_conv False
block1a_project_bn False
block1a_project_activation False
block1a_add False
block1b_project_conv False
block1b_project_bn False
block1b_project_activation False
block1b_drop False
block1b_add False
block1c_project_conv False
block1c_project_bn False
block1c_project_activation False
block1c_drop False
block1c_add False
block1d_project_conv False
block1d_project_bn False
block1d_project_activation False
block1d_drop False
block1d_add False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_project_conv False
block2b_project_bn False
block2b_drop False
block2b_add False
block2c_expand_conv False
block2c_expand_bn False
block2c_expand_activation False
block2c_project_conv False
block2c_project_bn False
block2c_dr

In [12]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 10

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 11/25
563/564 [============================>.] - ETA: 0s - loss: 1.2161 - accuracy: 0.6572
Epoch 11: val_loss did not improve from 0.91656
564/564 [==============================] - 91s 144ms/step - loss: 1.2161 - accuracy: 0.6572 - val_loss: 1.0563 - val_accuracy: 0.7177 - lr: 0.0010
Epoch 12/25
563/564 [============================>.] - ETA: 0s - loss: 1.1486 - accuracy: 0.6745
Epoch 12: val_loss did not improve from 0.91656
564/564 [==============================] - 79s 140ms/step - loss: 1.1486 - accuracy: 0.6745 - val_loss: 0.9280 - val_accuracy: 0.7452 - lr: 0.0010
Epoch 13/25
563/564 [============================>.] - ETA: 0s - loss: 1.0990 - accuracy: 0.6942
Epoch 13: val_loss did not improve from 0.91656
564/564 [==============================] - 79s 140ms/step - loss: 1.0991 - accuracy: 0.6942 - val_loss: 0.9322 - val_accuracy: 0.7340 - lr: 0.0010
Epoch 14/25
563/564 [============================>.] - ETA: 0s - loss: 1.0491 - accuracy: 0.7079
Epoch 14: val_loss improved

In [13]:
len(base_model.layers)

1028

In [14]:

for layer in base_model.layers[-150:]:
    layer.trainable = True


In [15]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 25

model1.fit(train_dataset,
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 26/40
563/564 [============================>.] - ETA: 0s - loss: 0.7499 - accuracy: 0.7937
Epoch 26: val_loss did not improve from 0.53341
564/564 [==============================] - 90s 144ms/step - loss: 0.7499 - accuracy: 0.7937 - val_loss: 0.6899 - val_accuracy: 0.8145 - lr: 0.0010
Epoch 27/40
563/564 [============================>.] - ETA: 0s - loss: 0.7755 - accuracy: 0.7946
Epoch 27: val_loss did not improve from 0.53341
564/564 [==============================] - 79s 140ms/step - loss: 0.7757 - accuracy: 0.7946 - val_loss: 0.6266 - val_accuracy: 0.8222 - lr: 0.0010
Epoch 28/40
563/564 [============================>.] - ETA: 0s - loss: 0.7618 - accuracy: 0.7966
Epoch 28: val_loss did not improve from 0.53341
564/564 [==============================] - 79s 140ms/step - loss: 0.7617 - accuracy: 0.7966 - val_loss: 0.7033 - val_accuracy: 0.8366 - lr: 0.0010
Epoch 29/40
563/564 [============================>.] - ETA: 0s - loss: 0.7504 - accuracy: 0.8011
Epoch 29: val_loss did not 

In [16]:
for layer in base_model.layers[-250:]:
    layer.trainable = True


In [17]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 40

model1.fit(train_dataset,
                                       
                                        validation_data=validation_dataset,
                                       
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 41/55
563/564 [============================>.] - ETA: 0s - loss: 0.6159 - accuracy: 0.8318
Epoch 41: val_loss did not improve from 0.44794
564/564 [==============================] - 97s 150ms/step - loss: 0.6160 - accuracy: 0.8318 - val_loss: 0.5327 - val_accuracy: 0.8580 - lr: 0.0010
Epoch 42/55
563/564 [============================>.] - ETA: 0s - loss: 0.6423 - accuracy: 0.8318
Epoch 42: val_loss did not improve from 0.44794
564/564 [==============================] - 82s 146ms/step - loss: 0.6422 - accuracy: 0.8318 - val_loss: 0.5945 - val_accuracy: 0.8493 - lr: 0.0010
Epoch 43/55
563/564 [============================>.] - ETA: 0s - loss: 0.6281 - accuracy: 0.8366
Epoch 43: val_loss did not improve from 0.44794
564/564 [==============================] - 82s 146ms/step - loss: 0.6282 - accuracy: 0.8366 - val_loss: 0.6245 - val_accuracy: 0.8484 - lr: 0.0010
Epoch 44/55
563/564 [============================>.] - ETA: 0s - loss: 0.6092 - accuracy: 0.8459
Epoch 44: val_loss did not 

In [18]:
for layer in base_model.layers[-300:]:
    layer.trainable = True

model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 70

model1.fit(train_dataset,
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 71/85
563/564 [============================>.] - ETA: 0s - loss: 0.5220 - accuracy: 0.8611
Epoch 71: val_loss did not improve from 0.38999
564/564 [==============================] - 102s 161ms/step - loss: 0.5219 - accuracy: 0.8612 - val_loss: 0.4563 - val_accuracy: 0.8846 - lr: 0.0010
Epoch 72/85
563/564 [============================>.] - ETA: 0s - loss: 0.5396 - accuracy: 0.8634
Epoch 72: val_loss did not improve from 0.38999
564/564 [==============================] - 80s 142ms/step - loss: 0.5398 - accuracy: 0.8633 - val_loss: 0.4823 - val_accuracy: 0.8897 - lr: 0.0010
Epoch 73/85
563/564 [============================>.] - ETA: 0s - loss: 0.5322 - accuracy: 0.8656
Epoch 73: val_loss did not improve from 0.38999
564/564 [==============================] - 80s 141ms/step - loss: 0.5321 - accuracy: 0.8657 - val_loss: 0.5279 - val_accuracy: 0.8867 - lr: 0.0010
Epoch 74/85
563/564 [============================>.] - ETA: 0s - loss: 0.5521 - accuracy: 0.8676
Epoch 74: val_loss did not

In [19]:
for layer in base_model.layers[-400:]:
    layer.trainable = True

model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 85

model1.fit(train_dataset,
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 86/100
563/564 [============================>.] - ETA: 0s - loss: 0.4644 - accuracy: 0.8736
Epoch 86: val_loss did not improve from 0.33496
564/564 [==============================] - 96s 148ms/step - loss: 0.4643 - accuracy: 0.8736 - val_loss: 0.4313 - val_accuracy: 0.8899 - lr: 0.0010
Epoch 87/100
563/564 [============================>.] - ETA: 0s - loss: 0.4928 - accuracy: 0.8762
Epoch 87: val_loss did not improve from 0.33496
564/564 [==============================] - 81s 143ms/step - loss: 0.4927 - accuracy: 0.8763 - val_loss: 0.4610 - val_accuracy: 0.9178 - lr: 0.0010
Epoch 88/100
563/564 [============================>.] - ETA: 0s - loss: 0.5201 - accuracy: 0.8791
Epoch 88: val_loss did not improve from 0.33496
564/564 [==============================] - 81s 143ms/step - loss: 0.5200 - accuracy: 0.8791 - val_loss: 0.4287 - val_accuracy: 0.8984 - lr: 0.0010
Epoch 89/100
563/564 [============================>.] - ETA: 0s - loss: 0.5477 - accuracy: 0.8727
Epoch 89: val_loss did 

KeyboardInterrupt: 

In [20]:
for layer in base_model.layers[-850:]:
    layer.trainable = True

model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 100

model1.fit(train_dataset,
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 101/115
563/564 [============================>.] - ETA: 0s - loss: 0.4605 - accuracy: 0.8840
Epoch 101: val_loss did not improve from 0.33496
564/564 [==============================] - 95s 146ms/step - loss: 0.4606 - accuracy: 0.8839 - val_loss: 0.4799 - val_accuracy: 0.8804 - lr: 0.0010
Epoch 102/115
563/564 [============================>.] - ETA: 0s - loss: 0.5084 - accuracy: 0.8791
Epoch 102: val_loss did not improve from 0.33496
564/564 [==============================] - 80s 142ms/step - loss: 0.5086 - accuracy: 0.8790 - val_loss: 0.4052 - val_accuracy: 0.9086 - lr: 0.0010
Epoch 103/115
563/564 [============================>.] - ETA: 0s - loss: 0.4901 - accuracy: 0.8876
Epoch 103: val_loss did not improve from 0.33496
564/564 [==============================] - 79s 141ms/step - loss: 0.4901 - accuracy: 0.8876 - val_loss: 0.5062 - val_accuracy: 0.9173 - lr: 0.0010
Epoch 104/115
563/564 [============================>.] - ETA: 0s - loss: 0.4782 - accuracy: 0.8895
Epoch 104: val_l

KeyboardInterrupt: 

2024-03-22 22:58:52.479915: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.38MiB (rounded to 9830400)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-03-22 22:58:52.480243: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-03-22 22:58:52.480252: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 506, Chunks in use: 506. 126.5KiB allocated for chunks. 126.5KiB in use in bin. 40.6KiB client-requested in use in bin.
2024-03-22 22:58:52.480257: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 217, Chunks in use: 217. 134.5KiB allocated for chunks. 134.5KiB in use in bin. 118.2KiB client-requested in use in bin.
2024-03-22 22:58:52.480260: I tensorflow/tsl/fram

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 